# MFCC Features and Simple Classifiers

In [41]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, 
    sklearn.svm, sklearn.ensemble, sklearn.linear_model
from sklearn.metrics import f1_score

import librosa
import librosa.display
from IPython import display

METADATA_PATH = 'dataset/FMA/fma_metadata/'

In [26]:
def load_metadata(path):
    tracks = pd.read_csv(os.path.join(METADATA_PATH, "tracks.csv"), index_col=0, header=[0, 1])
    keep_cols = [('set', 'split'), ('set', 'subset'), ('track', 'genre_top')]
    tracks = tracks[keep_cols]
    
    features = pd.read_csv(os.path.join(METADATA_PATH, "features.csv"), index_col=0, header=[0, 1, 2], skip_blank_lines=True)
    return tracks, features

def setup_data(tracks, features):
    small = tracks['set', 'subset'] == 'small'
    
    train = tracks['set', 'split'] == 'training'
    val = tracks['set', 'split'] == 'validation'
    test = tracks['set', 'split'] == 'test'
    
    y_train = tracks.loc[small & train, ('track', 'genre_top')]
    y_val = tracks.loc[small & val, ('track', 'genre_top')]
    y_test = tracks.loc[small & test, ('track', 'genre_top')]

    X_train = features.loc[small & train, 'mfcc']
    X_val = features.loc[small & val, 'mfcc']
    X_test = features.loc[small & test, 'mfcc']
    
    # Shuffle training data
    X_train, y_train = skl.utils.shuffle(X_train, y_train)
    
    # Standardize features - remove mean and scale accordingly
    standardize = skl.preprocessing.StandardScaler(copy=False)
    X_train = standardize.fit_transform(X_train)
    X_val = standardize.fit_transform(X_val)
    X_test = standardize.fit_transform(X_test)
    
    # Label encode outputs
    le = skl.preprocessing.LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_val = le.fit_transform(y_val)
    y_test = le.fit_transform(y_test)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

def train(c, X_train, y_train):
    c.fit(X_train, y_train)

def predict(c, X_test):
    y_pred = c.predict(X_test)
    return y_pred

In [ ]:
tracks, features = load_metadata(METADATA_PATH)

In [24]:
X_train, y_train, X_val, y_val, X_test, y_test = setup_data(tracks, features)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

['Electronic' 'Experimental' 'Folk' 'Hip-Hop' 'Instrumental'
 'International' 'Pop' 'Rock']
(6400, 140)
(800, 140)
(800, 140)


In [37]:
# SVC
c_svc = skl.svm.SVC()
train(c_svc, X_train, y_train)
y_pred_train = predict(c_svc, X_train)
y_pred_val = predict(c_svc, X_val)
y_pred_test = predict(c_svc, X_test)
print("SVC - F1 scores:")
print("Train: {:.4f}".format(f1_score(y_train, y_pred_train, average='micro', pos_label=1)))
print("Valid: {:.4f}".format(f1_score(y_val, y_pred_val, average='micro', pos_label=1)))
print("Test: {:.4f}".format(f1_score(y_test, y_pred_test, average='micro', pos_label=1)))

SVC - F1 scores:
Train: 0.7581
Valid: 0.5450
Test: 0.4650


In [42]:
# Random Forests
c_rf = skl.ensemble.RandomForestClassifier(random_state=10, max_depth=30, n_estimators=300, min_samples_leaf=6, min_impurity_decrease=0.0002, 
                                           class_weight='balanced')
train(c_rf, X_train, y_train)
y_pred_train = predict(c_rf, X_train)
y_pred_val = predict(c_rf, X_val)
y_pred_test = predict(c_rf, X_test)
print("Random Forests - F1 scores:")
print("Train: {:.4f}".format(f1_score(y_train, y_pred_train, average='micro', pos_label=1)))
print("Valid: {:.4f}".format(f1_score(y_val, y_pred_val, average='micro', pos_label=1)))
print("Test: {:.4f}".format(f1_score(y_test, y_pred_test, average='micro', pos_label=1)))

Random Forests - F1 scores:
Train: 0.9250
Valid: 0.5225
Test: 0.4437


In [44]:
# Logistic Regression
c_lr = skl.linear_model.LogisticRegression()
train(c_lr, X_train, y_train)
y_pred_train = predict(c_lr, X_train)
y_pred_val = predict(c_lr, X_val)
y_pred_test = predict(c_lr, X_test)
print("Logistic Regression - F1 scores:")
print("Train: {:.4f}".format(f1_score(y_train, y_pred_train, average='micro', pos_label=1)))
print("Valid: {:.4f}".format(f1_score(y_val, y_pred_val, average='micro', pos_label=1)))
print("Test: {:.4f}".format(f1_score(y_test, y_pred_test, average='micro', pos_label=1)))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression - F1 scores:
Train: 0.5620
Valid: 0.4863
Test: 0.4025
